## One Battery Model Scheduling

This Julia code implements an optimization model for electric bus scheduling using Mixed-Integer Linear Programming (MILP). The primary goal is to minimize the total number of buses required to cover a set of scheduled trips while considering charging time constraints.

### 1. Import necessary libraries for data handling and optimization

In [ ]:
using DataFrames, CSV
using JuMP, Gurobi, Cbc

### 2. --- Data Loading ---

In [ ]:
# Read the bus schedule from a CSV file into a DataFrame
table = CSV.read("OCCT_Schedule_OneBatteryV3.csv", DataFrame);

In [ ]:
# Get the total number of scheduled trips (rows in the table)
nn = size(table)[1]

### 3. --- Parameters & Constants ---

In [ ]:
supply = 80      # Energy supply/charging rate per kW
alpha = 0.2      # Lower bound of battery state of charge (SoC)
beta = 0.8;      # Upper bound of battery state of charge (SoC)
charge = zeros(1, nn) # Array to store required ready-times for each trip
h = 0;           # Buffer or constant adjustment for charging time

### 4. --- Optimization Model Setup ---

In [ ]:
# Initialize the optimization model using the Gurobi solver
m = Model(Gurobi.Optimizer)

# Define Decision Variables:
# x[i, j] is a binary variable (0 or 1)
# x[i, j] = 1 if trip 'i' is followed by trip 'j' by the same bus.
# The index nn+1 represents a virtual "depot" or "end of day" state.
@variable(m, x[1:nn+1, 1:nn+1], Bin)

# --- Objective Function ---
# Minimize the number of transitions to the "depot" (nn+1).
# Essentially, this minimizes the total number of buses used.
@objective(m, Min, sum(x[nn+1, j] for j = 1:nn+1))

# --- Scheduling & Connectivity Constraints ---

# 1. Self-loop prevention: A trip cannot be followed by itself
for i = 1:nn+1
    @constraint(m, x[i, i] == 0)
end

# 2. Chronological Ordering:
# A trip cannot connect to a trip that occurred earlier in time.
# This prevents traveling backward in time in the schedule.
for i = 2:nn 
    for j = 1:i-1
         @constraint(m, x[i, j] == 0) 
    end
end                           

# 3. Outflow Constraint:
# Every trip 'i' must have exactly one successor (either another trip or the depot).
for i = 1:nn
        @constraint(m, sum(x[i, j] for j = 1:nn+1) == 1) 
end

# 4. Inflow Constraint:
# Every trip 'j' must have exactly one predecessor.
for j = 1:nn
        @constraint(m, sum(x[i, j] for i = 1:nn+1) == 1) 
end                            

# --- Charging & Timing Constraints ---

# Calculate the 'ready time' for each trip. 
# It assumes column 4 of the table contains the end time + required charge time.
for i = 1:nn
    charge[i] = table[i, 4] + h
end                               

# 5. Temporal Feasibility:
# If bus connects trip 'i' to trip 'j' (x[i,j]=1), the start time of trip 'j' 
# (table[j, 3]) must be greater than or equal to the time trip 'i' is ready.
for i = 1:nn-1
     for j = 1+i:nn 
            @constraint(m, table[j, 3] >= x[i, j] * charge[i])
     end
end                                             

# --- Solve & Output ---
# Run the optimization solver
status = JuMP.optimize!(m)

# Display the minimum number of buses required
println("Objective value: ", JuMP.objective_value(m))

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3721 rows, 3721 columns and 10921 nonzeros
Model fingerprint: 0xfa37a760
Variable types: 0 continuous, 3721 integer (3721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 60.0000000
Presolve removed 3614 rows and 2185 columns
Presolve time: 0.01s
Presolved: 107 rows, 1536 columns, 3027 nonzeros
Found heuristic solution: objective 6.0000000
Variable types: 0 continuous, 1536 integer (1536 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 12 (of 12 available processors)

Solution count 2: 6 60 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%

User-callback 